In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("kmetashot_exp_classEval2.csv", header = 0, index_col=0)
df.head()

,bin,ass2ref,taxid,species,genus,family,order,class,phylum,superkingdom,organism_name,name,taxid_exp,species_exp,genus_exp,E_genus,E_species,E_taxid
0,Achromobacter_piechaudii_ATCC_43553_uid46343.f...,1.000000,742159,72556,222,506,80840,28216,1224,2,Achromobacter piechaudii ATCC 43553,Achromobacter_piechaudii_ATCC_43553_uid46343.f...,True,True,True,222,72556,742159
1,Achromobacter_xylosoxidans_C54_uid38739.fna.gz,1.000000,562971,85698,222,506,80840,28216,1224,2,Achromobacter xylosoxidans C54,Achromobacter_xylosoxidans_C54_uid38739.fna.gz,True,True,True,222,85698,562971
2,Acidaminococcus_D21_uid34117.fna.gz,1.000000,2970351,187327,904,909930,1843488,909932,1239,2,Acidaminococcus intestini,Acidaminococcus_D21_uid34117.fna.gz,False,True,True,904,187327,187327
3,Acinetobacter_ATCC_27244_uid30949.fna.gz,1.000000,525244,525244,469,468,2887326,1236,1224,2,Acinetobacter sp. ATCC 27244,Acinetobacter_ATCC_27244_uid30949.fna.gz,True,True,True,469,525244,525244
4,Acinetobacter_RUH2624_uid38511.fna.gz,0.996759,575564,575564,469,468,2887326,1236,1224,2,Acinetobacter sp. RUH 2624,Acinetobacter_RUH2624_uid38511.fna.gz,True,False,True,469,106654,575564


In [3]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score, f1_score

In [4]:
df.shape

(939, 18)

In [5]:
df.describe()

,ass2ref,taxid,species,genus,family,order,class,phylum,superkingdom,E_genus,E_species,E_taxid
count,939.000000,9.390000e+02,9.390000e+02,9.390000e+02,9.390000e+02,9.390000e+02,9.390000e+02,939.000000,939.00000,9.390000e+02,9.390000e+02,9.390000e+02
mean,0.909868,8.782349e+05,2.009947e+05,4.284265e+05,2.723475e+05,2.829444e+05,1.701019e+05,42227.937167,6.56230,3.670563e+05,1.667458e+05,7.046397e+05
std,0.248781,6.045114e+05,4.469689e+05,8.104100e+05,6.214935e+05,5.065341e+05,2.987413e+05,83941.158625,99.40418,7.601256e+05,2.739941e+05,2.011543e+05
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.00000,7.100000e+01,7.200000e+01,2.870000e+02
25%,1.000000,5.562595e+05,1.305000e+03,8.480000e+02,1.300000e+03,8.500900e+04,1.760000e+03,1224.000000,2.00000,8.480000e+02,1.315500e+03,5.470440e+05
50%,1.000000,7.495440e+05,2.702000e+03,1.485000e+03,3.395800e+04,1.868020e+05,9.106100e+04,1239.000000,2.00000,1.485000e+03,2.812900e+04,7.427300e+05
75%,1.000000,9.046865e+05,1.627770e+05,2.397590e+05,1.868030e+05,1.868260e+05,2.006430e+05,1239.000000,2.00000,1.868020e+05,2.116650e+05,8.830805e+05
max,1.000000,2.974241e+06,2.914161e+06,2.862142e+06,2.937909e+06,2.887326e+06,1.737404e+06,508458.000000,2157.00000,2.767893e+06,2.107999e+06,1.639133e+06


In [6]:
def score_metrics(tp, fn ,tn, fp):
    sensitivity = tp/(tp+fn)
    precision = tp/(tp+fp)
    specificity = tn/(tn+fp)
    f1_score = (2*sensitivity*precision)/(sensitivity+precision)
    fpr = fp/(fp+tn)
    ba = (sensitivity + specificity)/2
    accuracy = (tp+tn)/sum([tp, fn ,tn, fp])
    return sensitivity, specificity, precision, f1_score, fpr, ba, accuracy

In [7]:
# ass2ref, taxid, species, genus, family, E_genus,E_species, E_taxid 
estrain_tot, especies_tot, egenus_tot = 64320, 37639, 3713
data_list = []
for soglia in range(0,105,5):
    soglia /= 100
    Assigned = []
    # true positive (tp) sets for strain, species and genera.
    ttp, stp, gtp = [], [], []
    # false positive (fp) sets for strain, species and genera.
    tfp, sfp, gfp = [], [], []
    # true negative (tn) sets for strain, species and genera.
    for genome in df.bin.values:
        ass2ref = float(df.loc[df.bin == genome, "ass2ref"].iloc[0])
        taxid, species, genus = str(df.loc[df.bin == genome, "taxid"].iloc[0]),str(df.loc[df.bin == genome, "species"].iloc[0]),str(df.loc[df.bin == genome, "genus"].iloc[0])
        etaxid, especies, egenus = str(df.loc[df.bin == genome, "E_taxid"].iloc[0]),str(df.loc[df.bin == genome, "E_species"].iloc[0]),str(df.loc[df.bin == genome, "E_genus"].iloc[0])
        # print(genome, taxid, species, genus)
        # print(genome, ass2ref,soglia)
        if ass2ref >= soglia:
            if taxid == etaxid:
                ttp.append(etaxid)
                Assigned.append("taxid")
            else:
                if taxid != "0": 
                    tfp.append(taxid)
                    Assigned.append("Uncorrect taxid")
            if species == especies:
                stp.append(especies)
                Assigned.append("species")
            else:
                if species != "0": 
                    sfp.append(species)
                    Assigned.append("Uncorrect species")
            if genus == egenus:
                gtp.append(egenus)
                Assigned.append("genus")
            else:
                if genus != "0": 
                    gfp.append(genus)
                    Assigned.append("Uncorrect genus")
        else:
            if genus == egenus:
                gtp.append(egenus)
                Assigned.append("genus")
            else:
                if genus != "0": 
                    gfp.append(genus)
                    Assigned.append("Uncorrect genus")
    tfn = list(df.E_taxid.values.astype("str"))
    for i in ttp: tfn.remove(i)
    sfn = list(df.E_species.values.astype("str"))
    for i in stp: sfn.remove(i)
    gfn = list(df.E_genus.values.astype("str"))
    for i in gtp: gfn.remove(i)
    ttp, tfn, tfp = len(ttp), len(tfn), len(tfp)
    stp, sfn, sfp = len(stp), len(sfn), len(sfp)
    gtp, gfn, gfp = len(gtp), len(gfn), len(gfp)
    ttn, stn, gtn = estrain_tot - sum([ttp,  tfp, tfn]), especies_tot - sum([stp,  sfp, sfn]), egenus_tot - sum([gtp, gfp, gfn])
    data = ["Strain",soglia, ttp, tfn, ttn, tfp,Assigned.count("taxid"),Assigned.count("taxid") + Assigned.count("Uncorrect taxid")]
    data.extend(score_metrics(ttp, tfn, ttn, tfp))
    # print("Strain",soglia, ttp, tfn, ttn, tfp,  score_metrics(ttp, tfn, ttn, tfp))
    data_list.append(data)
    data = ["Species",soglia, stp, sfn, stn, sfp, Assigned.count("species"),Assigned.count("species") + Assigned.count("Uncorrect species")] 
    data.extend(score_metrics(stp, sfn, stn, sfp))
    data_list.append(data)
    # print("Species",soglia, stp, sfn, stn, sfp,  score_metrics(stp, sfn, stn, sfp))
    data = ["Genus",soglia, gtp, gfn, gtn, gfp, Assigned.count("genus"),Assigned.count("genus") + Assigned.count("Uncorrect genus")]
    data.extend(score_metrics(gtp, gfn, gtn, gfp))
    data_list.append(data)
    #print(soglia, Assigned.count("taxid"), Assigned.count("species"), Assigned.count("genus"))
    # print("Genus",soglia, gtp, gfn, gtn, gfp,  score_metrics(gtp, gfn, gtn, gfp))
    


In [8]:
df_total = pd.DataFrame(data_list, columns= ["Rank","Threshold","TP","FN","TN","FP","Correctly Classified MAGs","Assigned MAGs","Sensitivity","Specificity","Precision","F1-score","FPR","Balanced Accuracy","Accuracy"])

In [9]:
df_total.head(15)

,Rank,Threshold,TP,FN,TN,FP,Correctly Classified MAGs,Assigned MAGs,Sensitivity,Specificity,Precision,F1-score,FPR,Balanced Accuracy,Accuracy
0,Strain,0.00,770,169,63225,156,770,926,0.820021,0.997539,0.831533,0.825737,0.002461,0.908780,0.994947
1,Species,0.00,838,101,36612,88,838,926,0.892439,0.997602,0.904968,0.898660,0.002398,0.945020,0.994979
2,Genus,0.00,872,67,2720,54,872,926,0.928647,0.980534,0.941685,0.935121,0.019466,0.954591,0.967412
3,Strain,0.05,770,169,63237,144,770,914,0.820021,0.997728,0.842451,0.831085,0.002272,0.908875,0.995134
4,Species,0.05,834,105,36620,80,834,914,0.888179,0.997820,0.912473,0.900162,0.002180,0.943000,0.995085
5,Genus,0.05,872,67,2720,54,872,926,0.928647,0.980534,0.941685,0.935121,0.019466,0.954591,0.967412
6,Strain,0.10,770,169,63247,134,770,904,0.820021,0.997886,0.851770,0.835594,0.002114,0.908954,0.995289
7,Species,0.10,833,106,36629,71,833,904,0.887114,0.998065,0.921460,0.903961,0.001935,0.942590,0.995297
8,Genus,0.10,872,67,2720,54,872,926,0.928647,0.980534,0.941685,0.935121,0.019466,0.954591,0.967412
9,Strain,0.15,770,169,63254,127,770,897,0.820021,0.997996,0.858417,0.838780,0.002004,0.909009,0.995398


In [10]:
df_total.to_csv("kMetaShot_classification_rate.csv", sep = "\t", index=None, quoting=None)

In [11]:
df_not_expected = pd.read_excel("outref_taxid_species_genus_HMP.xlsx", header=0, date_format="str",sheet_name="taxid")
df_not_expected.head()

,Taxid,Rank
0,888051,strain
1,649743,strain
2,866774,strain
3,469585,strain
4,469589,strain


In [38]:
df_total[df_total.Rank == "Strain"][["Threshold","FPR","Balanced Accuracy","F1-score","Correctly Classified MAGs","Assigned MAGs"]]

,Threshold,FPR,Balanced Accuracy,F1-score,Correctly Classified MAGs,Assigned MAGs
0,0.00,0.002461,0.908780,0.825737,770,926
3,0.05,0.002272,0.908875,0.831085,770,914
6,0.10,0.002114,0.908954,0.835594,770,904
9,0.15,0.002004,0.909009,0.838780,770,897
12,0.20,0.001925,0.909048,0.841070,770,892
15,0.25,0.001783,0.909119,0.845225,770,883
18,0.30,0.001641,0.909190,0.849421,770,874
21,0.35,0.001467,0.909277,0.854606,770,863
24,0.40,0.001404,0.909309,0.856507,770,859
27,0.45,0.001325,0.909348,0.858896,770,854


In [39]:
df_total[df_total.Rank == "Species"][["Threshold","FPR","Balanced Accuracy","F1-score","Correctly Classified MAGs","Assigned MAGs"]]

,Threshold,FPR,Balanced Accuracy,F1-score,Correctly Classified MAGs,Assigned MAGs
1,0.00,0.002398,0.945020,0.898660,838,926
4,0.05,0.002180,0.943000,0.900162,834,914
7,0.10,0.001935,0.942590,0.903961,833,904
10,0.15,0.001744,0.942685,0.907407,833,897
13,0.20,0.001744,0.940023,0.904424,828,892
16,0.25,0.001608,0.937961,0.904501,824,883
19,0.30,0.001444,0.936445,0.905681,821,874
22,0.35,0.001253,0.934411,0.906770,817,863
25,0.40,0.001144,0.934465,0.908788,817,859
28,0.45,0.001063,0.933441,0.909091,815,854


Adesso valutiamo a livello di strain quali strain non attese sono classificate e come

In [37]:
with open("not_expected_taxa2class_rate.csv","w") as tmp:
    tmp.write("Threshold\tRank\tNot Expected\tClassified\n")
    for soglia in range(0,105,5):
        soglia /= 100
        for rank in ["strain", "species"]:
            strain = df_not_expected[df_not_expected.Rank == rank].Taxid.values
            print(soglia,rank,len(set(strain)), df[df.E_taxid.isin(strain) & (df.ass2ref >= soglia) & (df.ass2ref != 0)].shape[0])
            tmp.write(f"{soglia}\t{rank}\t{len(set(strain))}\t{df[df.E_taxid.isin(strain) & (df.ass2ref >= soglia) & (df.ass2ref != 0)].shape[0]}\n")
        print("###")

0.0 strain 97 86
0.0 species 61 45
###
0.05 strain 97 84
0.05 species 61 45
###
0.1 strain 97 76
0.1 species 61 42
###
0.15 strain 97 71
0.15 species 61 37
###
0.2 strain 97 67
0.2 species 61 37
###
0.25 strain 97 59
0.25 species 61 35
###
0.3 strain 97 51
0.3 species 61 29
###
0.35 strain 97 43
0.35 species 61 23
###
0.4 strain 97 39
0.4 species 61 20
###
0.45 strain 97 34
0.45 species 61 18
###
0.5 strain 97 31
0.5 species 61 16
###
0.55 strain 97 25
0.55 species 61 12
###
0.6 strain 97 20
0.6 species 61 9
###
0.65 strain 97 17
0.65 species 61 9
###
0.7 strain 97 16
0.7 species 61 8
###
0.75 strain 97 13
0.75 species 61 6
###
0.8 strain 97 12
0.8 species 61 6
###
0.85 strain 97 11
0.85 species 61 5
###
0.9 strain 97 7
0.9 species 61 2
###
0.95 strain 97 5
0.95 species 61 1
###
1.0 strain 97 3
1.0 species 61 1
###


## Performance Evaluation CAMITAX e GTDBtk

### CAMITAX

cm = pd.read_csv("camitax_exp_classEval2.csv", header=0, sep=",", index_col=0)
cm.head()

In [49]:
cm[["Genome","species","genus","E_species","E_genus"]]

,Genome,species,genus,E_species,E_genus
0,Achromobacter_piechaudii_ATCC_43553_uid46343,72556.0,222.0,72556,222
1,Achromobacter_xylosoxidans_C54_uid38739,85698.0,222.0,85698,222
2,Acidaminococcus_D21_uid34117,0.0,0.0,187327,904
3,Acinetobacter_ATCC_27244_uid30949,525244.0,469.0,525244,469
4,Acinetobacter_RUH2624_uid38511,106654.0,469.0,106654,469
...,...,...,...,...,...
934,Veillonella_parvula_ATCC_17745_uid41557,0.0,0.0,29466,29465
935,Veillonella_ratti_ACS_216_V_Col6b_uid52119,0.0,0.0,1502943,29465
936,Weissella_paramesenteroides_ATCC_33313_uid34661,1249.0,46255.0,1249,46255
937,Yokenella_regensburgei_ATCC_43003_uid65133,158877.0,158876.0,158877,158876


In [57]:
# ass2ref, taxid, species, genus, family, E_genus,E_species, E_taxid 
especies_tot, egenus_tot = 37639, 3713
data_list = []
Assigned = []
# true positive (tp) sets for species and genera.
stp, gtp = [], []
# false positive (fp) sets for species and genera.
sfp, gfp = [], []
# true negative (tn) sets for species and genera.
for genome in cm.Genome.values:
    species, genus = str(cm.loc[cm.Genome == genome, "species"].iloc[0]).split(".")[0],str(cm.loc[cm.Genome == genome, "genus"].iloc[0]).split(".")[0]
    especies, egenus = str(cm.loc[cm.Genome == genome, "E_species"].iloc[0]),str(cm.loc[cm.Genome == genome, "E_genus"].iloc[0])
    # print(genome, taxid, species, genus)
    # print(genome, ass2ref,soglia)
    if species == especies:
        stp.append(especies)
        Assigned.append("species")
    else:
        if species != "0": 
            sfp.append(species)
            Assigned.append("Uncorrect species")
    if genus == egenus:
        gtp.append(egenus)
        Assigned.append("genus")
    else:
        if genus != "0": 
            gfp.append(genus)
            Assigned.append("Uncorrect genus")
sfn = list(cm.E_species.values.astype("str"))
for i in stp: sfn.remove(i)
gfn = list(cm.E_genus.values.astype("str"))
for i in gtp: gfn.remove(i)
stp, sfn, sfp = len(stp), len(sfn), len(sfp)
gtp, gfn, gfp = len(gtp), len(gfn), len(gfp)
stn, gtn = especies_tot - sum([stp,  sfp, sfn]), egenus_tot - sum([gtp, gfp, gfn])

data = ["Species", stp, sfn, stn, sfp, Assigned.count("species"),Assigned.count("species") + Assigned.count("Uncorrect species")] 
data.extend(score_metrics(stp, sfn, stn, sfp))
data_list.append(data)
# print("Species",soglia, stp, sfn, stn, sfp,  score_metrics(stp, sfn, stn, sfp))

data = ["Genus", gtp, gfn, gtn, gfp, Assigned.count("genus"),Assigned.count("genus") + Assigned.count("Uncorrect genus")]
data.extend(score_metrics(gtp, gfn, gtn, gfp))
data_list.append(data)
#print(soglia, Assigned.count("taxid"), Assigned.count("species"), Assigned.count("genus"))
# print("Genus",soglia, gtp, gfn, gtn, gfp,  score_metrics(gtp, gfn, gtn, gfp))
cm_total = pd.DataFrame(data_list, columns= ["Rank","TP","FN","TN","FP","Correctly Classified MAGs","Assigned MAGs","Sensitivity","Specificity","Precision","F1-score","FPR","Balanced Accuracy","Accuracy"])

In [58]:
cm_total

,Rank,TP,FN,TN,FP,Correctly Classified MAGs,Assigned MAGs,Sensitivity,Specificity,Precision,F1-score,FPR,Balanced Accuracy,Accuracy
0,Species,651,288,36603,97,651,748,0.693291,0.997357,0.870321,0.771784,0.002643,0.845324,0.989771
1,Genus,701,238,2718,56,701,757,0.746539,0.979813,0.926024,0.826651,0.020187,0.863176,0.920819


### GTDB-tk

In [59]:
tk = pd.read_csv("gtdbtk_exp_classEval2.csv", header=0, sep=",", index_col=0)
tk.head()

,bins,superkingdom,phylum,class,order,family,genus,species,class@species,s__name,name,species_exp,genus_exp,E_genus,E_species
0,Achromobacter_piechaudii_ATCC_43553_uid46343,2,1224,1236,80840,119060,222,72556,True,Achromobacter piechaudii,Achromobacter_piechaudii_ATCC_43553_uid46343,True,True,222,72556
1,Achromobacter_xylosoxidans_C54_uid38739,2,1224,1236,80840,119060,222,85698,True,Achromobacter xylosoxidans,Achromobacter_xylosoxidans_C54_uid38739,True,True,222,85698
2,Acidaminococcus_D21_uid34117,2,1239,909932,1843488,909930,904,187327,True,Acidaminococcus intestini,Acidaminococcus_D21_uid34117,True,True,904,187327
3,Acinetobacter_ATCC_27244_uid30949,2,1224,1236,72274,468,469,29430,True,Acinetobacter haemolyticus,Acinetobacter_ATCC_27244_uid30949,False,True,469,525244
4,Acinetobacter_RUH2624_uid38511,2,1224,1236,72274,468,469,106654,True,Acinetobacter nosocomialis,Acinetobacter_RUH2624_uid38511,True,True,469,106654


In [60]:
# ass2ref, taxid, species, genus, family, E_genus,E_species, E_taxid 
especies_tot, egenus_tot = 37639, 3713
data_list = []
Assigned = []
# true positive (tp) sets for species and genera.
stp, gtp = [], []
# false positive (fp) sets for species and genera.
sfp, gfp = [], []
# true negative (tn) sets for species and genera.
for genome in tk.bins.values:
    species, genus = str(tk.loc[tk.bins == genome, "species"].iloc[0]).split(".")[0],str(tk.loc[tk.bins == genome, "genus"].iloc[0]).split(".")[0]
    especies, egenus = str(tk.loc[tk.bins == genome, "E_species"].iloc[0]),str(tk.loc[tk.bins == genome, "E_genus"].iloc[0])
    # print(genome, taxid, species, genus)
    # print(genome, ass2ref,soglia)
    if species == especies:
        stp.append(especies)
        Assigned.append("species")
    else:
        if species != "0": 
            sfp.append(species)
            Assigned.append("Uncorrect species")
    if genus == egenus:
        gtp.append(egenus)
        Assigned.append("genus")
    else:
        if genus != "0": 
            gfp.append(genus)
            Assigned.append("Uncorrect genus")
sfn = list(tk.E_species.values.astype("str"))
for i in stp: sfn.remove(i)
gfn = list(tk.E_genus.values.astype("str"))
for i in gtp: gfn.remove(i)
stp, sfn, sfp = len(stp), len(sfn), len(sfp)
gtp, gfn, gfp = len(gtp), len(gfn), len(gfp)
stn, gtn = especies_tot - sum([stp,  sfp, sfn]), egenus_tot - sum([gtp, gfp, gfn])

data = ["Species", stp, sfn, stn, sfp, Assigned.count("species"),Assigned.count("species") + Assigned.count("Uncorrect species")] 
data.extend(score_metrics(stp, sfn, stn, sfp))
data_list.append(data)
# print("Species",soglia, stp, sfn, stn, sfp,  score_metrics(stp, sfn, stn, sfp))

data = ["Genus", gtp, gfn, gtn, gfp, Assigned.count("genus"),Assigned.count("genus") + Assigned.count("Uncorrect genus")]
data.extend(score_metrics(gtp, gfn, gtn, gfp))
data_list.append(data)
#print(soglia, Assigned.count("taxid"), Assigned.count("species"), Assigned.count("genus"))
# print("Genus",soglia, gtp, gfn, gtn, gfp,  score_metrics(gtp, gfn, gtn, gfp))
tk_total = pd.DataFrame(data_list, columns= ["Rank","TP","FN","TN","FP","Correctly Classified MAGs","Assigned MAGs","Sensitivity","Specificity","Precision","F1-score","FPR","Balanced Accuracy","Accuracy"])

In [61]:
tk_total

,Rank,TP,FN,TN,FP,Correctly Classified MAGs,Assigned MAGs,Sensitivity,Specificity,Precision,F1-score,FPR,Balanced Accuracy,Accuracy
0,Species,731,208,36589,111,731,842,0.778488,0.996975,0.868171,0.820887,0.003025,0.887732,0.991525
1,Genus,805,134,2646,128,805,933,0.857295,0.953857,0.862808,0.860043,0.046143,0.905576,0.929437
